# Dynamic topic modelling via Non-negative Matrix Factorization (NMF)

In [1]:
! pip3 install gensim
! pip3 install razdel
! pip3 install pymorphy2

You are using pip version 18.0, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.0, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.0, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Import some preprocessing tools

In [2]:
import sys
sys.path.insert(0, '../../preprocessing') # /home/anya/project_news/proj_news_viz/nlp/preprocessing
import preprocessing_tools as preprocessing_tools

In [3]:
import pandas as pd;
import numpy as np;
import scipy as sp;
import sklearn;
import sys;
from nltk.corpus import stopwords;
import nltk;
from gensim.models import ldamodel
import gensim.corpora;
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer;
from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize;
import pickle;
import re;
from io import StringIO;
from nltk.corpus import stopwords

Here we will process input .csv line by line as far as the separator "|" happens to be a part of the message body

In [4]:
date  = []
url   = []
host  = []
topic = []
title = []
body  = []
with open('/home/anya/project_news/interfax_sorted_2008-02-11_2019-01-19.csv') as news_file:
    for line in news_file:
        fields = line.split("|")
        date.append(fields[0])
        url.append(fields[1])
        host.append(fields[2])
        topic.append(fields[3])
        title.append(fields[4])
        body.append(" ".join(fields[5:]))
data_prep = pd.DataFrame({'date' : date,
                          'url'  : url,
                         'host' : host,
                         'topic': topic,
                         'title': title,
                         'body' : body},
                         columns={'date', 'url', 'host', 'topic', 'title', 'body'})

In [5]:
data = pd.read_csv('/home/anya/project_news/interfax_sorted_2008-02-11_2019-01-19.csv',
                   sep='|',
                   encoding='utf-8',
                   error_bad_lines=False);
data.columns = ["date", "url", "host", "topic", "title", "body"]
data.info()

b'Skipping line 100300: expected 6 fields, saw 14\n'
b'Skipping line 138281: expected 6 fields, saw 7\nSkipping line 139718: expected 6 fields, saw 7\nSkipping line 184612: expected 6 fields, saw 7\nSkipping line 184720: expected 6 fields, saw 7\nSkipping line 188784: expected 6 fields, saw 8\nSkipping line 197770: expected 6 fields, saw 12\nSkipping line 201959: expected 6 fields, saw 7\nSkipping line 226778: expected 6 fields, saw 7\n'
b'Skipping line 332321: expected 6 fields, saw 8\nSkipping line 388909: expected 6 fields, saw 7\n'
b'Skipping line 393299: expected 6 fields, saw 7\nSkipping line 394088: expected 6 fields, saw 7\nSkipping line 396155: expected 6 fields, saw 7\nSkipping line 397254: expected 6 fields, saw 7\nSkipping line 397433: expected 6 fields, saw 10\nSkipping line 397542: expected 6 fields, saw 12\nSkipping line 398285: expected 6 fields, saw 14\nSkipping line 398351: expected 6 fields, saw 7\nSkipping line 401967: expected 6 fields, saw 9\nSkipping line 403339:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 529025 entries, 0 to 529024
Data columns (total 6 columns):
date     529025 non-null object
url      529025 non-null object
host     529025 non-null object
topic    529025 non-null object
title    529025 non-null object
body     529025 non-null object
dtypes: object(6)
memory usage: 24.2+ MB


In [6]:
data["cleaned_text"] = data["body"].apply(preprocessing_tools.clean_text)
data[["cleaned_text"]].head()

,cleaned_text
0,барнаул февраля interfaxru первый городской ...
1,чита февраля interfaxru эксглава нк \юкос\ м...
2,москва февраля interfaxru в результате пожар...
3,токио февраля interfaxru рост ввп россии в ...
4,москва февраля interfaxru цены на нефть выро...


In [7]:
texts = data["cleaned_text"].tolist()
texts = [re.sub(r'\\', '', x) for x in texts]

data["cleaned_text"] = data["cleaned_text"].apply(lambda x: re.sub(r'\\', '', x))
data[["cleaned_text"]].head()

,cleaned_text
0,барнаул февраля interfaxru первый городской ...
1,чита февраля interfaxru эксглава нк юкос мих...
2,москва февраля interfaxru в результате пожар...
3,токио февраля interfaxru рост ввп россии в ...
4,москва февраля interfaxru цены на нефть выро...


In [8]:
stopword_ru = stopwords.words('russian')
data["tokenized_text"] = data["cleaned_text"].apply(preprocessing_tools.lemmatization, stopword=stopword_ru)
data[["tokenized_text"]].head()

,tokenized_text
0,"[барнаул, февраль, interfaxru, городской, чемп..."
1,"[чита, февраль, interfaxru, эксглава, нк, юкос..."
2,"[москва, февраль, interfaxru, результат, пожар..."
3,"[токио, февраль, interfaxru, рост, ввп, россия..."
4,"[москва, февраль, interfaxru, цена, нефть, выр..."


In [14]:
tockens_clean = data["tokenized_text"].tolist()
texts_clean = [" ".join(x) for x in tockens_clean]

In [18]:
vectorizer = CountVectorizer(analyzer='word', max_features=3000);
x_counts = vectorizer.fit_transform(texts_clean);
print(x_counts.shape)

(529025, 3000)


In [19]:
transformer = TfidfTransformer(smooth_idf=False);
x_tfidf = transformer.fit_transform(x_counts);

In [20]:
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)

In [21]:
#obtain a NMF model.
num_topics = 200
model = NMF(n_components=num_topics, init='nndsvd');

#fit the model
model.fit(xtfidf_norm)

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0,
  max_iter=200, n_components=200, random_state=None, shuffle=False,
  solver='cd', tol=0.0001, verbose=0)

In [22]:
def get_nmf_topics(model, n_top_words):
    
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [23]:
topic_model = get_nmf_topics(model, 20)

In [24]:
topic_model

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 90,Topic # 91,Topic # 92,Topic # 93,Topic # 94,Topic # 95,Topic # 96,Topic # 97,Topic # 98,Topic # 99
0,журналист,произойти,рубль,матч,адвокат,пожар,самолёт,индекс,япония,край,...,вертолёт,космический,осетия,ноябрь,январь,полиция,решение,место,декабрь,правительство
1,сказать,инцидент,стоимость,тур,арест,возгорание,посадка,ммвб,японский,краснодарский,...,ми,аппарат,южный,москва,москва,полицейский,принять,занять,москва,премьерминистр
2,заявить,авария,составить,спартак,обвинение,пожарный,борт,пункт,остров,пермский,...,крушение,спутник,абхазия,interfaxru,interfaxru,отдел,окончательный,происшествие,interfaxru,вицепремьер
3,сообщить,пострадать,размер,цска,бывший,ликвидировать,полёт,ртс,аэс,красноярский,...,посадка,космодром,грузинский,месяц,новогодний,местный,связь,рейтинг,новогодний,заседание
4,песок,возгорание,триллион,стадион,следствие,тушение,пилот,рынок,вода,пермь,...,борт,орбита,северный,мх,австралия,стрельба,комиссия,третье,пермь,киргизия
5,пресссекретарить,причина,сумма,зенит,обвинять,огонь,крушение,фондовый,иена,краевой,...,пилот,запуск,независимость,конец,ск,столичный,принятие,занимать,пермский,распоряжение
6,interfaxru,предварительный,днём,го,предъявить,лесной,воздушный,торг,река,хабаровский,...,поиск,блок,эдуард,начало,снег,порядок,мера,балл,ск,кабинет
7,вицепремьер,происшествие,составлять,минута,стража,гореть,ан,биржа,уровень,губернатор,...,спасатель,байконур,признание,состояться,месяц,главка,власть,очки,конец,премьер
8,сми,обстоятельство,штраф,футбол,ходатайство,кв,разбиться,мск,авария,краснодар,...,разбиться,роскосмос,граница,марш,открытый,доставить,отменить,первое,состояться,постановление
9,слово,данные,доход,болельщик,пресечение,возникнуть,вылететь,вырасти,вод,приморский,...,катастрофа,вывести,конфликт,объявить,домодедово,умвд,обжаловать,работать,прессконференция,временной
